<a href="https://colab.research.google.com/github/anmolpandeybtech/Skin_Disease_Hackathon/blob/main/Skin_disease_vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import Javascript

def disable_colab_idle_timeout():
    js_code = '''
    function ClickConnect(){
        console.log("Working");
        document.querySelector("colab-toolbar-button#connect").click()
    }
    setInterval(ClickConnect,60000)
    '''
    display(Javascript(js_code))

disable_colab_idle_timeout()

<IPython.core.display.Javascript object>

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d ascanipek/skin-diseases

100% 6.17G/6.18G [00:55<00:00, 202MB/s]
100% 6.18G/6.18G [00:55<00:00, 119MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/skin-diseases.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
from keras import backend
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [6]:
input_size = (224, 224, 3)

base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_size)
for layer in base_model.layers:
    layer.trainable = False

80134624/80134624 [==============================] - 0s 0us/step


In [7]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
num_classes = 6  # Replace with the number of skin disease classes
predictions = Dense(num_classes, activation='softmax')(x)

In [8]:
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
batch_size = 32
num_epochs = 15

In [16]:
train_generator = train_datagen.flow_from_directory(
    '/content/kaggle/train',
    target_size=input_size[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/kaggle/val',  # Replace with the path to your validation dataset
    target_size=input_size[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/content/kaggle/test',
    target_size=input_size[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 30909 images belonging to 6 classes.
Found 3923 images belonging to 6 classes.
Found 3928 images belonging to 6 classes.


In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [18]:
checkpoint = ModelCheckpoint('best_skin_disease_vgg19.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=num_epochs,
    callbacks=[checkpoint]
)

Epoch 1/15
965/965 [==============================] - ETA: 0s - loss: 1.3243 - accuracy: 0.4486
Epoch 1: val_accuracy improved from -inf to 0.48745, saving model to best_skin_disease_vgg19.h5
965/965 [==============================] - 616s 624ms/step - loss: 1.3243 - accuracy: 0.4486 - val_loss: 1.2225 - val_accuracy: 0.4874


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/15
965/965 [==============================] - ETA: 0s - loss: 1.1959 - accuracy: 0.5031
Epoch 2: val_accuracy improved from 0.48745 to 0.52382, saving model to best_skin_disease_vgg19.h5
965/965 [==============================] - 599s 620ms/step - loss: 1.1959 - accuracy: 0.5031 - val_loss: 1.1973 - val_accuracy: 0.5238
Epoch 3/15
965/965 [==============================] - ETA: 0s - loss: 1.1394 - accuracy: 0.5295
Epoch 3: val_accuracy improved from 0.52382 to 0.53663, saving model to best_skin_disease_vgg19.h5
965/965 [==============================] - 585s 606ms/step - loss: 1.1394 - accuracy: 0.5295 - val_loss: 1.1458 - val_accuracy: 0.5366
Epoch 4/15
965/965 [==============================] - ETA: 0s - loss: 1.1037 - accuracy: 0.5462
Epoch 4: val_accuracy improved from 0.53663 to 0.56506, saving model to best_skin_disease_vgg19.h5
965/965 [==============================] - 587s 608ms/step - loss: 1.1037 - accuracy: 0.5462 - val_loss: 1.1057 - val_accuracy: 0.5651
Epoch 5/15